求最短距离的思路可以分成如下3步：   

1. 设置目标节点的attr=0，其余节点设置为无穷大 attr=Inf   
2. 从目标节点逆向往外传播 attr+1   
3. 每个节点对所有邻居消息以及当前的节点的attr取min,更新atrr  

重复2～3直到收敛，流程图大概如下：   
![avatar](./pic/最短距离.png)

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@746f5564
sc = org.apache.spark.SparkContext@528f02d9


org.apache.spark.SparkContext@528f02d9

In [2]:
val initGraph=util.GraphGenerators.rmatGraph(sc,32,64).
                           removeSelfEdges().
                           mapVertices((vid,_)=>{
                               val sourId=16L  //目标节点设置为16
                               if(vid==sourId) 0.0 else Double.PositiveInfinity
                           })

initGraph = org.apache.spark.graphx.impl.GraphImpl@72eddc52


org.apache.spark.graphx.impl.GraphImpl@72eddc52

In [3]:
initGraph.vertices.take(32)

Array((16,0.0), (22,Infinity), (28,Infinity), (30,Infinity), (24,Infinity), (18,Infinity), (20,Infinity), (26,Infinity), (19,Infinity), (21,Infinity), (25,Infinity), (29,Infinity), (27,Infinity), (23,Infinity), (17,Infinity), (31,Infinity))

In [4]:
val finalGraph=Pregel(initGraph,initialMsg=Double.PositiveInfinity,maxIterations=32)(
    vprog=(_,vd,aggMsg)=>math.min(vd,aggMsg),
    sendMsg=(edge)=>if(edge.dstAttr!=Double.PositiveInfinity) Iterator((edge.srcId,edge.dstAttr+1)) else Iterator.empty,
    mergeMsg=(a,b)=>math.min(a,b)
)

finalGraph = org.apache.spark.graphx.impl.GraphImpl@2c7d89f4


org.apache.spark.graphx.impl.GraphImpl@2c7d89f4

In [5]:
//计算结果
finalGraph.vertices.take(32)

Array((16,0.0), (22,1.0), (28,2.0), (30,2.0), (24,2.0), (18,1.0), (20,1.0), (26,1.0), (19,2.0), (21,3.0), (25,1.0), (29,Infinity), (27,2.0), (23,2.0), (17,2.0), (31,3.0))

In [6]:
//官方结果
lib.ShortestPaths.run(initGraph,Array(16L)).vertices.take(32)

Array((16,Map(16 -> 0)), (22,Map(16 -> 1)), (28,Map(16 -> 2)), (30,Map(16 -> 2)), (24,Map(16 -> 2)), (18,Map(16 -> 1)), (20,Map(16 -> 1)), (26,Map(16 -> 1)), (19,Map(16 -> 2)), (21,Map(16 -> 3)), (25,Map(16 -> 1)), (29,Map()), (27,Map(16 -> 2)), (23,Map(16 -> 2)), (17,Map(16 -> 2)), (31,Map(16 -> 3)))